Fasih Ahmed Khan (CS-23090)
Umar Saleem (CS-23042)

In [1]:
import tkinter as tk
from tkinter import ttk
import time
import threading
import random
from IPython.display import display, clear_output

In [2]:
N = 8

# Utility functions
def attacks(state):
    n = len(state)
    conflicts = 0
    for i in range(n):
        for j in range(i+1, n):
            if state[i] == state[j] or abs(state[i]-state[j]) == abs(i-j):
                conflicts += 1
    return conflicts

def random_state(n=N):
    return [random.randrange(n) for _ in range(n)]

# Hill climbing live
def hill_climb_live(initial_state, gui_callback, sleep_time=0.3):
    n = N
    current = initial_state[:]
    current_conf = attacks(current)
    restarts = 0

    while restarts < 50:
        if current_conf == 0:
            gui_callback(current)
            return current
        improved = False
        best_move = None
        best_conf = current_conf

        for col in range(n):
            orig_row = current[col]
            for row in range(n):
                if row == orig_row:
                    continue
                cand = current[:]
                cand[col] = row
                c_conf = attacks(cand)
                if c_conf < best_conf:
                    best_conf = c_conf
                    best_move = (col, row, cand)

        if best_move:
            col, row, candidate = best_move
            current = candidate
            current_conf = best_conf
            gui_callback(current)
            time.sleep(sleep_time)
            continue

        restarts += 1
        current = random_state(n)
        current_conf = attacks(current)
        gui_callback(current)
        time.sleep(sleep_time)

    return None

# CSP Backtracking live
def csp_backtrack_live(gui_callback, sleep_time=0.3):
    domains = {col: set(range(N)) for col in range(N)}
    assignment = {}

    def is_consistent(col, row):
        for c, r in assignment.items():
            if r == row or abs(r-row) == abs(c-col):
                return False
        return True

    def backtrack():
        if len(assignment) == N:
            return True
        var = min([c for c in range(N) if c not in assignment], key=lambda c: len(domains[c]))
        for val in sorted(domains[var]):
            if is_consistent(var, val):
                assignment[var] = val
                partial = [-1]*N
                for k,v in assignment.items():
                    partial[k] = v
                gui_callback(partial)
                time.sleep(sleep_time)
                if backtrack():
                    return True
                del assignment[var]
        return False

    result = backtrack()
    if not result:
        return None
    return [assignment[c] for c in range(N)]


In [3]:
class QueensGUI:
    def __init__(self, root):
        self.root = root
        root.title("8-Queens Solver - Live Simulation")
        self.canvas = tk.Canvas(root, width=400, height=400)
        self.canvas.grid(row=0, column=0, rowspan=20)

        ttk.Label(root, text="Select Algorithm").grid(row=0, column=1)
        self.algo = ttk.Combobox(root, values=["Hill Climbing", "CSP Backtracking"])
        self.algo.current(0)
        self.algo.grid(row=1, column=1)

        ttk.Label(root, text="Speed").grid(row=2, column=1)
        self.speed = ttk.Scale(root, from_=0.05, to=1.0, value=0.3, orient="horizontal")
        self.speed.grid(row=3, column=1)

        ttk.Button(root, text="Run", command=self.run_simulation).grid(row=5, column=1)
        ttk.Button(root, text="Reset", command=self.reset_board).grid(row=6, column=1)

        self.state = random_state()
        self.draw_board(self.state)

    def draw_board(self, state):
        self.canvas.delete("all")
        cell = 50
        for r in range(N):
            for c in range(N):
                color = "#EEE" if (r+c)%2==0 else "#777"
                self.canvas.create_rectangle(c*cell,r*cell,(c+1)*cell,(r+1)*cell, fill=color)
                if state[c]==r:
                    self.canvas.create_text(c*cell+25,r*cell+25,text="♛", font=("Arial",30), fill="red")

    def gui_update(self, state):
        self.root.after(0, lambda: self.draw_board(state))

    def run_bg(self, func):
        thread = threading.Thread(target=func)
        thread.daemon = True
        thread.start()

    def run_simulation(self):
        algo = self.algo.get()
        speed = float(self.speed.get())

        if algo == "Hill Climbing":
            init = random_state()
            self.state = init
            self.run_bg(lambda: hill_climb_live(init, self.gui_update, speed))
        elif algo == "CSP Backtracking":
            self.run_bg(lambda: csp_backtrack_live(self.gui_update, speed))

    def reset_board(self):
        self.state = random_state()
        self.draw_board(self.state)


In [6]:
import time

def hill_climb_metrics(initial_state):
    current = initial_state[:]
    current_conf = attacks(current)
    restarts = 0
    iterations = 0
    start = time.time()
    
    while restarts < 50:
        if current_conf == 0:
            return current_conf, True, restarts, time.time()-start
        
        best_move = None
        best_conf = current_conf
        for col in range(N):
            orig_row = current[col]
            for row in range(N):
                if row == orig_row:
                    continue
                cand = current[:]
                cand[col] = row
                c_conf = attacks(cand)
                if c_conf < best_conf:
                    best_conf = c_conf
                    best_move = (col,row,cand)
        
        if best_move:
            col,row,candidate = best_move
            current = candidate
            current_conf = best_conf
            iterations += 1
            continue
        
        restarts += 1
        current = random_state()
        current_conf = attacks(current)
    
    return current_conf, False, restarts, time.time()-start

def csp_metrics():
    assignment_count = 0
    def backtrack(assignment={}):
        nonlocal assignment_count
        if len(assignment) == N:
            return True
        var = min([c for c in range(N) if c not in assignment],
                  key=lambda c: N)  # domains fixed
        for val in range(N):
            conflict = False
            for c,r in assignment.items():
                if r==val or abs(r-val)==abs(c-var):
                    conflict=True
                    break
            if not conflict:
                assignment[var]=val
                assignment_count+=1
                if backtrack(assignment):
                    return True
                del assignment[var]
        return False
    
    start=time.time()
    success = backtrack()
    duration=time.time()-start
    return success, assignment_count, duration


In [7]:
import pandas as pd

trials = 5  # number of experiments
data = []

for t in range(1,trials+1):
    init_state = random_state()
    init_conf = attacks(init_state)
    
    hc_conf, hc_success, restarts, hc_time = hill_climb_metrics(init_state)
    csp_success, csp_nodes, csp_time = csp_metrics()
    
    data.append({
        "Trial": t,
        "Initial Conflicts": init_conf,
        "HC Success": hc_success,
        "Restarts": restarts,
        "HC Time": f"{hc_time:.3f} sec",
        "CSP Success": csp_success,
        "CSP Nodes": csp_nodes,
        "CSP Time": f"{csp_time:.3f} sec"
    })

df = pd.DataFrame(data)
df


,Trial,Initial Conflicts,HC Success,Restarts,HC Time,CSP Success,CSP Nodes,CSP Time
0,1,9,True,10,0.104 sec,True,113,0.008 sec
1,2,10,True,4,0.069 sec,True,113,0.007 sec
2,3,7,True,2,0.025 sec,True,113,0.011 sec
3,4,5,True,4,0.048 sec,True,113,0.008 sec
4,5,6,True,15,0.138 sec,True,113,0.005 sec


In [8]:
# Notebook me GUI ko thread me run karna
def run_gui():
    root = tk.Tk()
    app = QueensGUI(root)
    root.mainloop()

# Start GUI
threading.Thread(target=run_gui, daemon=True).start()
